# COMP9418 - Assignment 1 - Bayesian Networks as Classifiers

## UNSW Sydney, September 2021

- Dezhao Chen - z5302273
- Ziqiao Ringgold Lin - z5324329

## Instructions

**Submission deadline:** Sunday, 17th October 2021, at 18:00:00.

**Late Submission Policy:** The penalty is set at 20% per late day. This is ceiling penalty, so if a group is marked 60/100 and they submitted two days late, they still get 60/100.

**Form of Submission:** This is a group assignment. Each group can have up to **two** students. **Only one member of the group should submit the assignment**.

You can reuse any piece of source code developed in the tutorials.

Submit your files using give. On a CSE Linux machine, type the following on the command-line:

``$ give cs9418 ass1 solution.zip``

Alternative, you can submit your solution via [WebCMS](https://webcms3.cse.unsw.edu.au/COMP9418/21T3).

## Technical prerequisites

These are the libraries your are allowed to use. No other libraries will be accepted. Make sure you are using Python 3.

In [180]:
# Allowed libraries
import numpy as np
import pandas as pd
import scipy as sp
import heapq as pq
import matplotlib as mp
import math
from itertools import product, combinations
from graphviz import Digraph
from tabulate import tabulate
import copy

We can also use the python files we developed in tutorials, or any other code from the tutorials.

In [181]:
from DiscreteFactors import Factor
from Graph import Graph
from BayesNet import BayesNet            

## Initial task - Initialise graph

Create a graph ``G`` that represents the following network by filling in the edge lists.
![Bayes Net](BayesNet.png)


In [182]:
G = Graph({
    "BreastDensity" : ["Mass"],
    "Location" : ["BC"],
    "Age" : ["BC"],
    "BC" : ["Metastasis","MC","SkinRetract","NippleDischarge","AD","Mass"],
    "Mass" : ["Size","Shape","Margin"],
    "AD" : ["FibrTissueDev"],
    "Metastasis" : ["LymphNodes"],
    "MC" : [],
    "Size" : [],
    "Shape" : [],
    "FibrTissueDev" : ["SkinRetract","NippleDischarge","Spiculation"],
    "LymphNodes" : [],
    "SkinRetract" : [],
    "NippleDischarge" : [],
    "Spiculation" : ["Margin"],
    "Margin" : [],
})

### Load data

In [183]:
# load data
with open('bc.csv') as file:
    data = pd.read_csv(file)

#remove 2 variables from data (because we are pretending we don't know this information)
if 'Metastasis' in data:
    del data['Metastasis']
if 'LymphNodes' in data:
    del data['LymphNodes']

# remove same 2 nodes from graph
G.remove_node('Metastasis')
G.remove_node('LymphNodes')

## [20 Marks] Task 1 - Efficient d-separation test

Implement the efficient version of the d-separation algorithm in a function ``d_separation(G, X, Z, Y)`` that return a boolean: ``True`` if **X** is d-separated from **Y** given **Z** in the graph $G$ and ``False`` otherwise.

* **X**,**Y** and **Z** are python sets, each containing a set of variable names. 
* Variable names may be strings or integers, and can be assumed to be nodes of the graph $G$. 
* $G$ is a directed graph object as defined in tutorial 1.

In [184]:
def d_separation(G, X, Z, Y):
    '''
    Arguments:
    G:   is an object of type Graph (the class you developed in tutorial 1)
    X,Z and Y:  are python `set` objects.
    '''
    # Step 0: Make a deep copy of the gragh G in case we need to reuse the same graph later
    G_dict = copy.deepcopy(G.adj_list)
    G_copy = Graph(G_dict)

    # Step 1: Keep deleting any leaf nodes from G if it's not in X∪Y∪Z
    set_XYZ = (X.union(Y)).union(Z)
    while True:
        deletable_leafs = []
        for node in G_copy.adj_list.keys():
            # Get all deletable leaves for current G
            if node not in set_XYZ and len(G_copy.adj_list[node]) == 0:
                deletable_leafs.append(node)
        if len(deletable_leafs) == 0:
            # Stop the leaf deletion after no more isolated leaves found
            break
        else:
            # Remove the deletable leaves from G in this round
            for node in deletable_leafs:
                G_copy.remove_node(node)
    # print("After Step 1:", G_copy.adj_list)

    # Step 2: Remove any outgoing edges from node(s) Z
    for node in Z:
        G_copy.adj_list[node] = []
    # print("After Step 2:", G_copy.adj_list)

    # Step 3: Make current G undirected
    for key in G_copy.adj_list.keys():
        for node in G_copy.adj_list[key]:
            if key not in  G_copy.adj_list[node]:
                G_copy.adj_list[node].append(key)
    # print("After Step 3:", G_copy.adj_list)

    # Step 4: Check the connectivity from X to Y in current graph G
    overall_connected = False
    for node in X:
        connectivity_color_map = G_copy.dfs(node)
        # print(connectivity_color_map)
        for dest in Y:
            if connectivity_color_map[dest] == 'black' or connectivity_color_map[dest] == 'grey':
                # black or grey indicates this node is found in DFS search for at least once
                overall_connected = True
    return not overall_connected

In [185]:
# Custom tests
d_separation(G, set(['Age']), set(['BC', 'MC']), set(['AD']))

True

In [186]:
############
## TEST CODE
## Note: More hidden tests will be used. You should make more tests yourself.

def test(statement):
    if statement:
        print("Passed test case")
    else:
        print("Failed test case")
        
test(d_separation(G, set(['Age']), set(['BC']), set(['AD'])))
test(not d_separation(G, set(['Spiculation','SkinRetract']), set(['MC', 'Size']), set(['Age'])))

Passed test case
Passed test case


## [10 Marks] Task 2 - Estimate Bayesian Network parameters from data

Implement a function ``learn_outcome_space(data)`` that learns the outcome space (the valid values for each variable) from the pandas dataframe ``data`` and returns a dictionary ``outcomeSpace`` with these values.

Implement a method ``model.learn_parameters(data, alpha=1)`` that learns the parameters of the Bayesian Network `model`. This function should do the same as the ``learn_parameters`` function from tutorials, but it should also implement laplacian smoothing with parameter $\alpha$.

In [ ]:
## Develop your code for learn_outcome_space(data) in one or more cells here

In [187]:
def learn_outcome_space(data):
    '''
    Arguments:
        data - A pandas dataframe
    Returns: 
        outcomeSpace - A dictionary. e.g. {'A':('True', 'False'), 'B':('up','down','left'), 'C':(1,2,3,4)}
    '''
    outcomeSpace = {}
    for col in data.columns:
        outcomeSpace[col] = tuple(data[col].unique())
    return outcomeSpace


outcomeSpace = learn_outcome_space(data)

In [188]:
############
## TEST CODE

outcomeSpace = learn_outcome_space(data)

outcomes = outcomeSpace['BreastDensity']
answer = ('high', 'medium', 'low')
test(len(outcomes) == len(answer) and set(outcomes) == set(answer))

Passed test case


In [189]:
## Develop your code for learnParameters in one or more cells here
def allEqualThisIndex(dict_of_arrays, **fixed_vars):
    # base index is a boolean vector, everywhere true
    first_array = dict_of_arrays[list(dict_of_arrays.keys())[0]]
    index = np.ones_like(first_array, dtype=np.bool_)
    for var_name, var_val in fixed_vars.items():
        index = index & (np.asarray(dict_of_arrays[var_name])==var_val)
    return index

def estimateFactor(data, var_name, parent_names, outcomeSpace,alpha):
    var_outcomes = outcomeSpace[var_name]
    parent_outcomes = [outcomeSpace[var] for var in (parent_names)]
    # cartesian product to generate a table of all possible outcomes
    all_parent_combinations = product(*parent_outcomes)

    f = Factor(list(parent_names)+[var_name], outcomeSpace)
    
    for i, parent_combination in enumerate(all_parent_combinations):
        parent_vars = dict(zip(parent_names, parent_combination))
        parent_index = allEqualThisIndex(data, **parent_vars)
        for var_outcome in var_outcomes:
            var_index = (np.asarray(data[var_name])==var_outcome)
            f[tuple(list(parent_combination)+[var_outcome])] = ((var_index & parent_index).sum()+alpha)/(parent_index.sum()+alpha*len(var_outcomes))
            
    return f

In [190]:
class BayesNet(BayesNet):
    def learn_parameters(self, data, alpha=1):
        graphT = self.graph.transpose()
        for node, parents in graphT.adj_list.items():
            f = estimateFactor(data, node, parents, self.outcomeSpace,alpha)
            self.factors[node] = f

model = BayesNet(G,outcomeSpace)

In [211]:
############
## TEST CODE

model.learn_parameters(data, alpha=1)

test(model.factors['Age']['35-49'] == 0.248000399920016)

Passed test case


## [20 Marks] Task 3 - Bayesian Network Classification

Design a new function ``assess_bayes_net(model, dataframe, var)`` that uses the test cases in ``dataframe`` to assess the performance of the Bayesian network at classifying the variable `var`. Implement the efficient classification procedure discussed in the lectures (make sure you understand what a Markov Blanket is). Such a function should return the classifier accuracy. 

 * ``var`` is the name of the variable you are predicting, using the values of all the other variables. 
 
If you like, you can add new functions to the BayesNet class, or write helper functions to help solve the above task.

Using another function called `cross_validation_bayes_net`, compute and report the average accuracy over the ten cross-validation runs as well as the standard deviation. A scaffold for this function is provided below.

In [ ]:
## Develop your code for assess_bayes_net in one or more cells here

In [345]:
outcomeSpace = learn_outcome_space(data)
model = BayesNet(G,outcomeSpace)
model.learn_parameters(data, alpha=1)

def get_parents(G):
    '''
    Create a parent nodes dict list for input graph and return it
    '''
    graph_parents_list={}
    for node in G:
        graph_parents_list[node]=[]
    for parent_node in G:
        for child_node in G[parent_node]:
            graph_parents_list[child_node].append(parent_node)
    return graph_parents_list

def get_markov_blanket(model: BayesNet, var: str):
    '''
    1. Find all parents
    2. Find all children
    3. Find all spouses
    '''
    markov_blanket = []
    graph = model.graph.adj_list
    # add children
    markov_blanket = markov_blanket + graph[var]
    # add parent
    graph_parents_list = get_parents(graph)
    markov_blanket = markov_blanket + graph_parents_list[var]
    
    # add all spouses
    for node in graph[var]:
        markov_blanket = markov_blanket + graph_parents_list[node]
    markov_blanket = list(dict.fromkeys(markov_blanket))
    
    # Make sure both verions of markov blankets are returned
    markov_blanket_without_var = copy.deepcopy(markov_blanket)
    if var in markov_blanket:
        markov_blanket_without_var.remove(var)
    else:
        markov_blanket.append(var)

    return markov_blanket, markov_blanket_without_var

def get_related_table(model: BayesNet, var_space: list):
    '''
    Get the joint table of a given bayes net model of a given variable space
    '''
    all_table = model.factors
    table = all_table[var_space[0]]
    for i in range(1, len(var_space)):
        table = table.join(all_table[var_space[i]])
    return table

def assess_bayes_net(model, dataframe, var='BC'):
    '''
    Test the accuracy given trained model by using the test dataframe
    '''
    # Get the probability table of all the variables in 'var' markov blanket
    markov_blanket, markov_blanket_without_var = get_markov_blanket(model, var)
    # markov_blanket_table = get_related_table(model, markov_blanket)
    related_test_table = dataframe[markov_blanket]
    correct_amount = 0
    total_amount = 0
    print(related_test_table.shape)
    for row_index, row_data in related_test_table.iterrows():
        var_value_actual = row_data.pop(var)
        row_data_dict = dict(row_data)
        query_result = model.query([var], **row_data_dict)
        query_result_values = query_result.table
        # print(query_result_values)
        var_possible_values = model.outcomeSpace[var]
        # print(var_possible_values)
        if len(query_result_values) != len(var_possible_values):
            print("Warning: query table not the same with actual values:", query_result_values, var_possible_values)
        possibility_max = np.max(query_result_values)
        possibility_max_index = np.argmax(possibility_max)
        # print("Guessed index: ", possibility_max_index)
        var_value_prediced = var_possible_values[possibility_max_index]
        # print("Guessed value: ", var_value_prediced)
        
        if var_value_prediced == var_value_actual:
            correct_amount += 1
        total_amount += 1
        # print(possibility_max, var_value_prediced)
        # break
    accuracy = correct_amount * 1.0 / (total_amount * 1.0)
    print(accuracy)
    return accuracy

# def cross_validation_bayes_net(dataframe, var='BC', k=10):
#     accuracy_list = []
#     for i in range(k):
#         # split dataset into train and test
#         ...
        
#         # train a model
#         ...
        
#         # test the model with assess_bayes_net
#         acc = ...
        
#         accuracy_list.append(acc)
#     return np.mean(accuracy_list), np.std(accuracy_list)


In [346]:
# Custom tests
print(assess_bayes_net(model, data, var='BC'))

(20000, 10)


In [ ]:
############
## TEST CODE

acc, stddev = cross_validation_bayes_net(data, 'BC', 10)
test(abs(acc - 0.85) < 0.05)

## [10 Marks] Task 4 - Naïve Bayes Classification

Design a new function ``assess_naive_bayes(model, data, var)`` to classify and assess the test cases in ``data``. To classify each example, use the log probability trick discussed in the lectures. Do $k$-fold cross-validation with the `cross_validation_naive_bayes(data, var, k)` function, same as above, and return ``acc`` and ``stddev``.

In [ ]:
## Develop your code for assess_naive_bayes(model, data, var) in one or more cells here

In [ ]:
...


def assess_naive_bayes(model, dataframe, var='BC'):
    ...
    return accuracy
    
def cross_validation_naive_bayes(dataframe, var='BC', k=10):
    accuracy_list = []
    for i in range(k):
        # split dataset into train and test
        ...
        
        # create and train a model
        ...
        
        # test the model with assess_naive_bayes
        acc = ...
        
        accuracy_list.append(acc)
    return np.mean(accuracy_list), np.std(accuracy_list)
    

In [ ]:
############
## TEST CODE

acc, stddev = cross_validation_naive_bayes(data, 'BC')
test(abs(acc - 0.80) < 0.05)

## [20 Marks] Task 5 - Tree-augmented Naïve Bayes Classification

Similarly to the previous task, implement a Tree-augmented Naïve Bayes (TAN) classifier and evaluate your implementation in the breast cancer dataset. Design a function ``learn_tan_structure(data, class_var)`` to learn the TAN structure (graph) from ``data`` and return such a structure. Scaffolds for required functions are given below. Implement other helper functions as necessary.

In [ ]:
## Develop your code for learn_tan_structure(data) in one or more cells here

In [ ]:
def learn_tan_structure(data, class_var='BC'):
    '''
    Arguments:
        data: a dataframe
        class_var: The variable you will be classifying with this graph structure
    Return:
        graph: A Graph object
    '''
    ...
    return graph

...

def cross_validation_tan(data, var='BC', k=10):
    ...
    return np.mean(accuracy_list), np.std(accuracy_list)

In [ ]:
############
## TEST CODE

tan_graph = learn_tan_structure(data)
test(len(tan_graph.children('BC')) == len(tan_graph)-1)
test('FibrTissueDev' in tan_graph.children('Spiculation') or 'Spiculation' in tan_graph.children('FibrTissueDev'))

## [20 Marks] Task 6 - Report

Write a report (**with less than 500 words**) summarising your findings in this assignment. Your report should address the following:

a. Make a summary and discussion of the experimental results. You can analyse your results from different aspects such as accuracy, runtime, coding complexity and independence assumptions. You can use plots to illustrate your results.

b. Discuss the time and memory complexity of the implemented algorithms.

Use Markdown and Latex to write your report in the Jupyter notebook. Develop some plots using Matplotlib to illustrate your results. Be mindful of the maximum number of words. Please, be concise and objective.